**Übung Mustererkennung** *WS 2022/23* -- *K. Brandenbusch,  Gernot A. Fink* -- *Technische Universität Dortmund, Lehrstuhl XII, Mustererkennung in eingebetteten Systemen*
___
# Aufgabe 10: Neuronale Netze mit PyTorch

In den letzten Jahren haben sich [PyTorch](https://pytorch.org/) und [Tensorflow](https://www.tensorflow.org/) als Frameworks für den Einsatz von neuronalen Netzen durchgesetzt.
Beide Frameworks implementieren eine Reihe von Datensätzen, Netzwerkschichten und Optimierern.
Desweiteren übernimmt die [autograd](https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html)-Funktion von PyTorch die automatische Definition und Berechnung von Gradienten.

In dieser Aufgabe entwickeln Sie ein simples neuronales Netz in PyTorch.

Zunächst muss das Notebook konfiguriert werden.

In [0]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

# Uebergeordneten Ordner zum Pfad hinzufuegen, damit das common Package importiert werden kann
import sys
if '..' not in sys.path:
    sys.path.append('..')

___
### Datesets und Dataloader

PyTorch bietet einen einheitlichen Standard für das Laden und Iterieren von Datensätzen.
Dazu stellt PyTorch den praktischen [`DataLoader`](https://pytorch.org/docs/stable/data.html) zur Verfügung.
Ein `DataLoader` iteriert über einen Datensatz und erstellt automatisch Batches mehrerer Samples.


Die meisten gängigen [Benchmark Datensätze](https://pytorch.org/vision/stable/datasets.html) (wie z.B. MNIST) sind direkt in PyTorch implementiert und können einfach importiert werden.
Die Datensatz-Klasse kümmert sich dabei auch um das Herunterladen der Daten.

In dieser Aufgabe befassen Sie sich mit einer vereinfachten Form des MNIST Datensatzes.
Wir reduzieren dabei den Datensatz auf 2 Klassen.


In [0]:
import torch
import torchvision.datasets as datasets
from torchvision import transforms

import numpy as np
import matplotlib.pyplot as plt

dataset = datasets.MNIST(root='../data/torchvision',
                        transform=transforms.ToTensor(),
                        train=True,
                        download=True)

# Extrahieren von zwei Klassen
idx = (dataset.targets == 0) | (dataset.targets == 1)
dataset.targets = dataset.targets[idx]
dataset.data = dataset.data[idx]

# DataLoader erstellen
batch_size = 10
train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Visualisierung einer Minibatch des ausgewählten Teils des MNIST Datensatzes, der klassifiziert werden soll
batch, target = next(iter(train_loader))
_, ax_arr = plt.subplots(2,5)
ax_arr = ax_arr.flatten()
for idx, img in enumerate(batch):
    ax_arr[idx].imshow(np.squeeze(img), cmap='Greys_r')
    ax_arr[idx].axis('off')
plt.show()

---
## Netzarchitektur
In den nächsten Schritten implementieren Sie das folgende Netz:

![CNN](./cnn.jpg)

### Konstruktor

Implementieren Sie zunächst den Konstruktor der Klasse `ConvolutionalNeuralNetwork` im Modul [`common.cnn`](../common/cnn.py).
Im Konstruktor der Klasse sollen zunächst nur die Bauteile des Netzes initialisiert werden.

Initialisieren Sie zwei Faltungsschichten
* conv1 := conv2d : in_channel=1, out_channel=10, kernel_size=5
* conv2 := conv2d : in_channel=10, out_channel=20, kernel_size=5

und zwei vollvernetzte Schichten
* fc1 := linear: in_features=$ 4 \times 4 \times 20$, out_features=10
* fc2 := linear: in_features=10, out_features=2

Für weitere Informationen lesen Sie die PyTorch Dokumentation:
* [conv2d](https://pytorch.org/docs/stable/nn.html#conv2d)
* [linear](https://pytorch.org/docs/stable/nn.html#linear)

Beantworten Sie zusätzlich die folgenden Fragen:

* Warum sollten MLPs nicht zur Klassifikation von Bildern verwendet werden?
* Warum ist es sinnvoll als Aktivierungsfunktion der versteckten Schichten die
Rectified Linear Unit zu verwenden?

In [0]:
from common.cnn import ConvolutionalNeuralNetwork
# Initialisierung des Netzes
cnn = ConvolutionalNeuralNetwork()

### Forward

Implementieren Sie nun die Methode `ConvolutionalNeuralNetwork.forward`.
Diese Methode definiert *wie* die Daten durch das Netz geführt werden.

Die Aktivierungsfunktion `ReLU` und das Pooling können Sie aus dem Modul `torch.nn.functional` verwenden.

Für weitere Informationen lesen Sie die PyTorch Dokumentation:
* [max-pool2d](https://pytorch.org/docs/stable/nn.functional.html#max-pool2d)
* [relu](https://pytorch.org/docs/stable/nn.functional.html#relu)
* [log_softmax](https://pytorch.org/docs/stable/generated/torch.nn.functional.log_softmax.html)
* [view](https://pytorch.org/docs/stable/generated/torch.Tensor.view.html)

---
### Evaluation
Implementieren Sie außerdem die Methode `ConvolutionalNeuralNetwork.test` um das Netz sowohl nach dem Training zu Evaluieren als auch um das Training des Netzes zu überwachen.

Die Methode soll die Ausgabe des Netzes für die übergebenen Daten berechnen und anschließend sowohl den Optimierungsfehler als auch die Genauigkeit der Vorhersage zurückgeben.

Nutzen Sie die negative log-likelihood [nll_loss](https://pytorch.org/docs/stable/generated/torch.nn.functional.nll_loss.html) als Optimierungsfehler.

Nutzen sie [`torch.no_grad()`](https://pytorch.org/docs/stable/generated/torch.no_grad.html) um die automatische Berechnung der Gradienten zu stoppen.



Nachdem Sie alle notwendigen Bestandteile der Klasse implementiert haben, trainiert und evaluiert der folgende Code ihr Netz.

In [0]:
import torch
import torch.nn.functional as F
import torch.optim as optim

learning_rate = 0.0001
log_interval = 10
test_interval = 100

optimizer = optim.Adam(cnn.parameters(), lr=learning_rate)

# Training des Netzes fure eine Epoche
cnn.train()
for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = cnn(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()

    if batch_idx % log_interval == 0:
        print('Iteration: [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    if batch_idx % test_interval == 0:
        print('Evaluatiere nach {} Iterationen...'.format(batch_idx *len(data)))
        test_loss, accuracy, correct = cnn.test(test_loader)
        print('Test set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
                test_loss, correct, len(test_loader.dataset),
                100 * accuracy))

print('Finale Evalauierung:')
test_loss, accuracy, correct = cnn.test(test_loader)
print('Test set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100 * accuracy))

---
**Optional**

Passen Sie das Modell für die Klassifikation aller MNIST Klassen an und versuchen Sie möglichst gute Ergebnisse auf dem Testdatensatz zu erreichen.  
Implemnetieren Sie beispielsweise das LeNet5.
Wenn Sie eine GPU zur Verfügung haben, können Sie auch andere (größere) [Architekturen aus PyTorch laden](https://pytorch.org/vision/stable/models.html#classification).

Experimentieren Sie mit anderen [Datensätzen](https://pytorch.org/vision/stable/datasets.html).